In [2]:
# -------------------------------------------------------- Module 1 
# ---------------------------- Introduction aux bases de données relationnelles

# -------------------- 1. Système de gestion de base de données relationnelles

# Un système de gestion de base de données (SGBD) est un logiciel permettant à un 
# utilisateurs d'accéder à une base de données

# -------------------- 2. Schémas 

# -------------------- 3. Clés 

# Exécutez cette cellule pour importer les fonctions que l'on va utiliser
from IPython.display import Markdown, display

# --------------------

# Exécuter la cellule pour afficher la réponse
display(Markdown("Utilisateurs ( <u>UtilisateurId</u> , Prenom, Nom , Pays de residence )"))

# --------------------

# Exécuter la cellule pour afficher la réponse
display(Markdown("Modules ( <u>ModuleId</u> , Nom , Difficulte )"))
display(Markdown("Utilisateurs (<u>UtilisateurId</u>, Nom, Prénom, #MetierId )")) 
display(Markdown("Metiers( <u>MetierId</u>, Nom )")) 
display(Markdown("L'attribut *MetierId* de la table **Utilisateurs** fait référence à la clé primaire de la table **Metiers**. ")) 

# --------------------

# Exécuter la cellule pour afficher la réponse

display(Markdown("Candidature(<u>#IdEtudiant</u>, Session, <u>#IdUniversite</u>, Adresse, Nom Parcours)"))

display(Markdown("* Un étudiant peut effectuer plusieurs candidatures dans une même université à des parcours différents"))
display(Markdown("* L’attribut **IdEtudiant** fait référence à la clé primaire de la table **Etudiants**"))
display(Markdown("* L’attribut **IdUniversite** fait référence à la clé primaire de la table **Universite**"))
display(Markdown("Etudiants(Nom, Prenom, Adresse, <u>IdEtudiant</u>)"))
display(Markdown("Universite (<u>IdUnversite</u>, NomUni, AdresseUniversite, Rang )"))

# -------------------- 4. Schéma relationnel 

# Exécuter la cellule pour afficher la réponse
display(Markdown('Relation entre Universite et Etudiants: One to many '))
display(Markdown('En effet, un étudiant  est affecte a une seule université  et une université  est elle associé à plusieurs étudiants .'))
display(Markdown('Relation entre Etudiant et Candidature : many to many '))
display(Markdown('En effet, un étudiant  peut soumettre sa candidature a plusieurs formations universitaire et une Candidature est unique elle est associé à un seul étudiant.'))

# --------------------

print("C'est la contrainte d'intégrité référentielle. En effet l'utilisateur 4 n'existe pas dans la table Utilisateurs.")

# -------------------- Conclusion
# 
# Ce qu'il faut retenir 
#Les clés primaires identifient de manière une unique un tuple d'une table.
#Les clés étrangères sont les clés primaires d'autres tables.
#Les clés peuvent être composites.
#Les différents schémas relationnels


In [ ]:
# -------------------------------------------------------- Module 2 
# -------------------------------------------------- Introduction à SQL Alchemy



# -------------------- 1. SQL Alchemy

# Exécutez cette cellule pour importer les fonctions que l'on va utiliser
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

# SQL Alchemy est une librairie Python très utilisée pour se connecter à un 
# système de gestion de bases de données relationnelles. 
# Il permet d'interpréter du code SQL

# --------------------

# En ajoutant l'argument echo = True, cela nous permet d'afficher la traduction
# en langage SQL des commandes du module
engine = create_engine('sqlite:///college.db', echo=True)

# --------------------

# Exécuter la cellule pour afficher la réponse
meta = MetaData()

# -------------------- Création de table

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key=True), 
   Column('name', String), 
   Column('lastname', String),
)
meta.create_all(engine)

# --------------------

parcours = Table(
   'parcours', meta, 
   Column('id_parcours', Integer, primary_key=True), 
   Column('name', String), 
   Column('student_id', Integer, ForeignKey("students.id")),
)
meta.create_all(engine)

# -------------------- 2. Transactions simples

# Une transaction ACID est une transaction qui respecte les points suivants:

# Atomicité : une transaction doit être effectuée en entier ou abandonnée, c'est à dire que si une opération est refusée, toutes les opérations de la transaction ne seront pas validées.
# Cohérente : l'état initial et final doivent être cohérents, c'est à dire respecter les contraintes d'intégrité.
# Isolation : Une transaction doit s'effectuer indépendamment des transactions qui s'exécutent en même temps, c'est à dire que les états intermédiaires d'une transaction sont cachés aux autres.
# Durabilité : lorsqu'une transaction est validée, ses effets ne peuvent être perdus.

# Exécuter la cellule pour afficher la réponse

values = [
    (1, "Jean", "Dubois"),
    (2, "Florent", "Desbiolles"),
    ]

# --------------------

# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=students.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

# --------------------

# Exécuter la cellule pour afficher la réponse

values_parcours = [(1, "Data engineering", 1),(2, "Data Science", 2) ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values_parcours[0]))
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            
            ins = ins.format(tablename=parcours.name, markers=markers)
            connection.execute(ins, values_parcours)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()
            
# --------------------

# Exécuter cette cellule
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM students;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM parcours;")
    print(results.fetchall())
    
# --------------------

# Exécuter la cellule pour afficher la réponse

sql = text('DROP TABLE IF EXISTS students;')
result = engine.execute(sql)

###Vérification que la table students n'existe plus en réutilisant le code d'insertion de tuple

values = [
    (1,"Jean", "Dubois"),
    (2,"Florent", "Desbiolles"),
    ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values[0])) 
            ins = 'INSERT INTO students VALUES ({markers})'
            ins = ins.format(tablename=students.name, markers=markers)
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

# -------------------- Conclusion
# la structure de connection à une base de données avec SQL Alchemy
# la notion de transaction ACID
# la syntaxe pour créer ou supprimer une table en SQL

In [ ]:
# -------------------------------------------------------- Module 3 
# ---------------------------- Les bases du langage SQL


# -------------------- Préliminaires
import sqlite3, sqlalchemy
from sqlalchemy import inspect, create_engine, text

engine = create_engine("sqlite:///chinook.db")




# -------------------- 1. Utilisation de l'inspecteur
#Une classe très pratique pour afficher des informations 
inspector = inspect(engine)

# --------------------
#afficher les nom des tables
inspector.get_table_names()

# --------------------
# get_columns de la classe Inspector permet d'afficher 
# les caractéristiques des différents attributs
inspector.get_columns(table_name='albums')

# --------------------
# La méthode get_foreign_keys de l'objet inspector permet 
# d'afficher les clés étrangères 
inspector.get_foreign_keys(table_name='albums')




# -------------------- 2. Requêtes simples

SELECT colonne1, colonne2 FROM table1;     #import les col 1 et 2
SELECT * FROM table1;     #import toutes les col
SELECT DISTINCT * FROM table1;     #import toutes les col sans les doublons
SELECT * FROM table1 LIMIT 10;     #import les col (10lignes)

# --------------------
conn = engine.connect()

result = conn.execute("SELECT Title FROM albums LIMIT 10;")
result.fetchall() 
#Cette commande renvoie tous les tuples de la table de l'objet ResultProxy



# -------------------- 3. Utilisation de WHERE
stmt = text( "SELECT * FROM albums WHERE albums.ArtistId = 1")
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text( "SELECT * FROM invoices WHERE Total > 20;")
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text( "SELECT * FROM invoices WHERE Total > 20 AND Total < 22;")
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text( "SELECT InvoiceId, BillingCountry, Total FROM invoices WHERE Total BETWEEN 10 AND 20;")
result = conn.execute(stmt)
result.fetchall()

# -------------------- Les expressions régulières
 stmt = text ( "SELECT * FROM tracks WHERE Name LIKE '%Wild%';")
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT tracks.Name, tracks.Composer FROM tracks WHERE Composer LIKE '%Freddie%' AND Composer LIKE '%Mercury%'" )
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT tracks.Name, tracks.Composer FROM tracks"
             " WHERE Name LIKE 'P%' AND ((Composer LIKE '%/%' AND Composer NOT LIKE '__/__')OR Composer LIKE '%-%') ")
result = conn.execute(stmt)
result.fetchall()

# -------------------- ORDER BY
# On peut simplement ajouter les noms des attributs à extraire les uns à la suite des autres séparés par une virgule
stmt = text ( "SELECT InvoiceId, CustomerId , Total FROM invoices ORDER BY Total DESC LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()




# -------------------- 4. Agrégation de requête
# on a aussi choisi de classer les résultats par montant total décroissant
stmt = text ( "SELECT invoices.CustomerId , SUM(Total) AS tot FROM invoices GROUP BY CustomerId ORDER BY tot DESC;")
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT BillingCountry, AVG(Total) AS PaysMoy FROM invoices GROUP BY BillingCountry " )
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT BillingCountry, MAX(Total) AS tot FROM invoices GROUP BY BillingCountry ORDER BY tot DESC " )
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT BillingCountry, COUNT(*) AS p FROM invoices GROUP BY BillingCountry ORDER BY p")
result = conn.execute(stmt)
result.fetchall()

# --------------------
# Chaque paire (PAYS, VILLE) existant dans la base est représentée.
stmt = text ( "SELECT BillingCountry,BillingCity, COUNT(InvoiceId) FROM invoices GROUP BY BillingCountry,BillingCity" )
result = conn.execute(stmt)
result.fetchall()

# --------------------HAVING
stmt = text ( "SELECT CustomerId, COUNT(CustomerId) AS tot FROM invoices GROUP BY CustomerId HAVING tot > 5 ORDER BY tot" )
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT CustomerId, SUM(Total) AS tot FROM invoices GROUP BY CustomerId HAVING tot > 20;" )
result = conn.execute(stmt)
result.fetchall()

# --------------------
stmt = text ( "SELECT CustomerId, SUM(Total) AS tot FROM invoices WHERE Total > 20 GROUP BY CustomerId " )
result = conn.execute(stmt)
result.fetchall()

# -------------------- Conclusion
#La commande SELECT retourne des enregistrements dans un tableau de résultat.
#La commande WHERE dans une requête SQL permet d’extraire les lignes d’une base de données qui respectent un prédicat.
#La commande ORDER BY permet de trier les lignes dans un résultat d’une requête SQL.
#La commande GROUP BY est utilisée en SQL pour grouper plusieurs résultats et utiliser une fonction de totaux sur un groupe de résultat.
#La commandeHAVING permet de filtrer en utilisant des fonctions telles que SUM(), COUNT(), AVG(), MIN() ou MAX() utilisées par la commande GROUP BY.
#L'opérateur LIKE est utilisé dans la clause WHERE des requêtes SQL. A l'aide de Wildcards, on peut rechercher des agencements de mots ou lettres dans les chaînes de caractères d'attributs.


In [ ]:
# -------------------------------------------------------- Module 4 
# -------------------------------------------------------- Langage SQL avancé

# Exécutez cette cellule pour charger les modules nécessaires
import sqlite3, sqlalchemy
from sqlalchemy import inspect, create_engine, text

# On utilise //// 4 slash pour accéder à un fichier par son chemin absolu
engine = create_engine("sqlite:///chinook.db")

# Création de connexion avec l'engine 
conn = engine.connect()


# -------------------- 1. Requêtes imbriquées

stmt = text("SELECT * FROM (SELECT CustomerId, SUM(Total) AS total FROM invoices GROUP BY CustomerId ORDER BY Total LIMIT 10) table_temp ORDER BY Total")
result = conn.execute(stmt)
result.fetchall()



# -------------------- 2. Jointures

SELECT * FROM table1 CROSS JOIN table2;
stmt = text("SELECT * FROM tracks  CROSS JOIN genres;")
result = conn.execute(stmt)
result.fetchall()

# -------------------- Jointure interne ou INNER JOIN

#Ce type de jointure va permettre de concaténer les tuples 
#des 2 tables deux à deux si une condition est satisfaite. 

stmt = text ( "SELECT * FROM artists INNER JOIN albums ON artists.ArtistId = albums.ArtistId LIMIT 10;" )
result = conn.execute(stmt)
result.fetchall()



# -------------------- Jointure à droite ou RIGHT JOIN
stmt =text ("SELECT * FROM (SELECT CustomerId, Total  FROM 'invoices' WHERE InvoiceId BETWEEN 21 AND 40) AS A "
            "LEFT JOIN (SELECT CustomerId, Total FROM 'invoices' "
            "WHERE InvoiceId BETWEEN 1 AND 20) AS B ON A.CustomerId=B.CustomerId")
result = conn.execute(stmt)
result.fetchall()


# -------------------- 3. Modifier une table
Modifier une ligne avec SET et UPDATE

On peut utiliser UPDATE et SET pour modifier une ou plusieurs ligne(s). Par exemple, si on veut modifier la colonne colonne1 pour qu'elle prenne la valeur nouvelle_valeur lorsque la colonne colonne2 vaut valeur, on peut écrire:

UPDATE table1
SET colonne1 = nouvelle_valeur WHERE colonne2 = valeur;



Supprimer des lignes avec DELETE
Sur le même principe, on peut supprimer les lignes pour lesquelles la colonne colonne2 vaut valeur en faisant:

DELETE FROM table WHERE colonne2 = valeur;


# --------------------Conclusion

# les différents types de jointure
# comment modifier des valeurs
# comment supprimer des lignes

L'examen est composé de 2 exercices. Le premier consiste à créer une base de données et de manipuler des informations sur cette base. Le dernier traite principalement des deux derniers cours. Il consiste à ce connecter à une base de données de films et de récupérer des informations de cette base. Les exercices sont indépendants l'un de l'autre. N'hésitez pas à contacter l'équipe DataScientest si vous rencontrez des problèmes sur help@datascientest.com

(a) Importer les modules et fonctions suivantes : sqlaclhemy, sqlite3, inspect, create_engine, text,Table, Column, Integer, String, ForeignKey, MetaData, Float.

In [ ]:
import sqlite3, sqlalchemy
from sqlalchemy import inspect, create_engine, text

In [ ]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect

1. Exercice 1¶
(a) Créer une base de données nommée Distributeur.db et créer une table Boissons(BoissonId, Nom, Contenance, Prix) en choisissant de manière adéquate la clé primaire. Il faudra justifier votre réponse.

In [ ]:
engine = create_engine("sqlite:///Distributeur.db", echo=True)

# -------------------- 

meta = MetaData()

Boisson = Table(
   'Boisson', meta, 
    Column('boissonID', Integer, primary_key=True),  #cle car unique 
   Column('Nom', String), 
   Column('Contenance', Integer), 
   Column('Prix', String),
)
meta.create_all(engine)

In [ ]:
#Examen SQL
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

engine = create_engine("sqlite:///Distributeur.db", echo=True)
meta = MetaData()
boissons = Table(
   'Boissons', meta, 
   Column('BoissonId', Integer, primary_key=True), 
   Column('Nom', String), 
   Column('Contenance', Integer),
   Column('prix', Integer),    
)
meta.create_all(engine)

(b) Ajouter les tuples (1, Coca, 33, 0.99) ; (2, Perrier, 100, 2) ; (3, Perrier, 33, 1), (4, Vittel, 150, 1.50) , (5, Badoit, 50, 2). Pour cela, il faudra gérer les exceptions qui peuvent être levées.

In [ ]:
values = [
    (1, "Coca", 33,0.99),
    (2, "Perrier", 100,2),
    (3, "Perrier", 33,1),
    (4, "Vittel", 150,1.50),
    (5, "Badoit", 50,2),
    ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values[0])) 
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            ins = ins.format(tablename="Boissons", markers=markers)
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

(c) Créer une table Facture(FactureId,BoissId,Heure). Il faudra faire référence à une clé primaire de la table Boissons.

In [ ]:
facture = Table(
   'Facture', meta, 
   Column('FactureId', Integer, primary_key=True), 
   Column('BoissId', Integer, ForeignKey("Boissons.BoissonId")), 
   Column('Heure', Integer),
)
meta.create_all(engine)

(d) Ajouter les tuples (1,1,7), (2,1,8) , (3,4,12) dans la table Facture.

In [ ]:
values = [(1,1,7), 
          (2,1,8) , 
          (3,4,12)]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values[0])) 
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            ins = ins.format(tablename=Facture, markers=markers)
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

(e) Peut-on supprimer la table Boissons ? Justifier votre réponse.

(f) Peut-on ajouter le tuple dans la table Boissons (1,Sprite,50,1) ? Justifier votre réponse.


In [ ]:
#non on peut pas supprimer la table boissons on violerait la règle de cohérence

In [ ]:
#non car on violerait la règle de la clé car la clé primaire existe déjà 

(g) Donner en langage SQL, la requête permettant de savoir quelles boissons du distributeur n'ont pas été vendu depuis sa mise en place, puis l'exécuter. (On peut utiliser NOT IN pour vérifier qu'une colonne n'appartient pas à une table.)

In [ ]:
conn = engine.connect()

stmt = text("select * from Boissons where BoissonId NOT IN (select BoissId from Facture)")

result = conn.execute(stmt)
result.fetchall()


(h) Donner en langage SQL, la requête permettant de savoir la proportion des différentes boissons vendues dans l'état actuel du distributeur, puis l'exécuter.

In [ ]:
stmt = text("select BoissId, count(*) from Facture group by BoissId")

result = conn.execute(stmt)
result.fetchall()

i) Supprimer les deux tables.

In [ ]:
stmt = text('DROP TABLE IF EXISTS Facutre;')
stmt1 = text('DROP TABLE IF EXISTS Boissons;')
result = conn.execute(stmt)
result = conn.execute(stmt1)

2. Exercice 2¶


La base de données newIMDB.db est déjà chargée dans votre espace de travail. Cette base de données regroupent les informations de tous les titres référencés sur le site IMDB.

   La base de données est composée des caractéristiques de films, séries TV ainsi que d'autres type de production.
(a) Connecter vous à la base de données newIMDB.db.

In [ ]:
engine = create_engine("sqlite:///newIMDB.db")


(b) Effectuer une première analyse de la base de données en déterminant les différentes tables, leurs attributs, les clés primaires associées aux tables ...

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
inspector.get_columns(table_name='crew')
inspector.get_foreign_keys(table_name='crew')
inspector.get_columns(table_name='episodes')
inspector.get_foreign_keys(table_name='episodes')
inspector.get_columns(table_name='people')
inspector.get_foreign_keys(table_name='people')
inspector.get_columns(table_name='titles')
inspector.get_foreign_keys(table_name='titles')
inspector.get_columns(table_name='ratings')
inspector.get_foreign_keys(table_name='ratings')

In [ ]:
#les diffrents attributs par table
print("CREW",inspector.get_columns(table_name='crew'), "\n")
print("EPISODES",inspector.get_columns(table_name='episodes'), "\n")
print("PEOPLE",inspector.get_columns(table_name='people'), "\n")
print("RATINGS",inspector.get_columns(table_name='ratings'), "\n")
print("TITLES",inspector.get_columns(table_name='titles'), "\n")

In [ ]:
CREW [{'name': 'title_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'person_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'category', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'job', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'characters', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}] 

EPISODES [{'name': 'episode_title_id', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'show_title_id', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'season_number', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'eposide_number', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}] 

PEOPLE [{'name': 'person_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'born', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'died', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}] 

RATINGS [{'name': 'title_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'rating', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'votes', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}] 

TITLES [{'name': 'title_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'type', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'primary_title', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'original_title', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'is_adult', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'premiered', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'ended', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'runtime_minutes', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'genres', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}] 


(c) Quels sont les titres les mieux notés dans la base de données ? On indiquera le nom, la note, et le nombre de votants. Pour que la sélection soit représentative, il faudra filtrer pour les titres ayant un nombre de votants > 5000. (On limitera les résultats aux 10 premières lignes.)

In [ ]:
conn = engine.connect()
stmt = text("""select primary_title, rating, votes 
                from ((SELECT * FROM ratings WHERE votes > 5000 ) as A\
                 inner join (SELECT * FROM titles) as B 
                 on A.title_id = B.title_id) 
                 order by rating desc 
                 limit 10;""")

#stmt = text("select title_id, rating, votes from ratings where votes >5000 order by rating desc limit 10;")
result = conn.execute(stmt)
result.fetchall()

(d) Quelle est la note moyenne des titres par catégorie, classer ce résultat par ordre décroissant des notes moyennes ? (On limitera les résultats aux 10 premières lignes.)

In [ ]:
stmt = text("select category, avg(rating) as moy from (SELECT * from crew as c inner join ratings as r on c.title_id = r.title_id) temp group by category  ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
stmt = text("select genres, avg(rating) as moy from (SELECT * from titles as t inner join ratings as r on t.title_id = r.title_id) temp group by genres ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

(e) Quelles sont les notes moyennes des titres associés aux différents directeurs/metteurs en scène/réalisateurs ... ? (On limitera les résultats aux 10 premières lignes.)

Le mot clé dans la base de données pour désigner ces groupes de personnes est : director.

In [ ]:
stmt = text("select category, avg(rating) as moy from (SELECT * from crew as c inner join ratings as r on c.title_id = r.title_id) temp group by category having category like '%director%' ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
# Insérez votre code ici
#stmt = text("select distinct(category) from crew;")
stmt = text("select temp.title_id, temp.category, temp.rating from ((select c.title_id, c.category from crew as c where category like '%director%') as A\
             inner join (select * from ratings ) as B on A.title_id=B.title_id) as temp  limit 10;")
#stmt = text("select category,person_id, avg(rating) as moy from (SELECT * from crew as c inner join ratings as r on c.title_id = r.title_id) temp group by category having category like '%director%' ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

(f) Combien y a t-il de titres par type ? (Utiliser l'attribut titleType)

In [ ]:
stmt = text("select t.type, count(*) from titles as t group by t.type;")
result = conn.execute(stmt)
result.fetchall()

(g) Quels sont les titres sur lesquels Quentin Tarantino a travaillé ? On précisera le nom (primary) et la note du titre.

In [ ]:
#les titres sur lesquels Quentin Tarantino a travaillé
#stmt1= text("select person_id from people where name like '%Quentin%' and name like '%Tarantino%'")
#stmt2 = text("select title_id from crew where person_id in (select person_id from people where name like '%Quentin%' and name like '%Tarantino%'")")
#code à tester à l'examen
stmt3 = text ("select primary_title, rating from (select title_id from crew where person_id in (select person_id from people where name like '%Quentin%' and name like '%Tarantino%')) as A\
             inner join ratings as r where A.title_id = r.title_id ")

#stmt = text("select primary_title, rating as moy from (SELECT * from crew as c inner join ratings as r on c.title_id = r.title_id) temp group by category  ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
stmt = text ("select primary_title, rating from ( ((select * from crew where person_id in \
              (select person_id from people where name like '%Quentin%' and name like '%Tarantino%')) as A)\
             inner join (select * from ratings) as B on A.title_id = B.title_id ) as C \
             inner join (select * from titles ) as D on C.title_id = D.title_id;")

#stmt = text("select primary_title, rating as moy from (SELECT * from crew as c inner join ratings as r on c.title_id = r.title_id) temp group by category  ORDER BY moy desc LIMIT 10;")
result = conn.execute(stmt)
result.fetchall()

(h) Quels sont les acteurs ou actrices ayant obtenus les meilleurs notes (>9) ? La variable category de la table crew détermine le métier de l'individu auquel le tuple est associé. (On limitera les résultats aux 10 premières lignes)

In [ ]:
stmt = text("select C.title_id, C.person_id, C.category , C.rating from ((select title_id, person_id, category from crew where category='actor' or category='actress') as A\
              inner join (select * from ratings where rating>9) as B on A.title_id=B.title_id) as C") 

#stmt=text("select name, rating, rating from (select title_id, person_id from crew where category='actor' or category=''actress)")
result = conn.execute(stmt)
result.fetchall()

(i) Quelle est la note moyenne des contenus pour les actrices et les acteurs ? Regrouper ces deux informations dans un seul tuple?

In [ ]:
stmt = text("select category, avg (rating) from crew  as c inner join (select * from ratings  ) as r on c.title_id = r.title_id  group by category having category='actor' or category='actress' ;")
result = conn.execute(stmt)
result.fetchall()

3. Question bonus
(a) Quels sont les directeurs/metteurs en scène/réalisateurs (si ils existent) qui ont obtenu une note supérieur à 9 pour tous leurs titres ? (On limitera les résultats aux 10 premières lignes.)
   Le mot clé dans la base de données pour désigner ces groupes de personnes est : director.

In [ ]:
stmt = text("""select name, rating from (select category, person_id, rating from crew  as c inner join 
            (select * from ratings where rating > 9 ) as r on c.title_id = r.title_id  where  category='director' Limit 10) 
            as tab1 inner join (select * from people) limit 10
            """)
result = conn.execute(stmt)
result.fetchall()